### Imports

In [4]:
### Standard imports
import os
import shutil
import numpy as np
import pandas as pd
import nibabel as nib
import tensorflow as tf
from glob import glob
from os.path import join


### Custom helpers
import sys
from helpers import *

### Grey masks

In [40]:
masks = glob("/Volumes/hd_4tb/masks/plip/*")
masks = [{
    "code": os.path.basename(mask).split("_")[0],
    "data": nib.load(mask).get_data(),
} for mask in masks]

def _in_mask(masks, x, y, z):
    result = dict()
    for mask in masks:
        code = mask["code"]
        data = mask["data"]
        result[code] = bool(data[x, y, z])
    return result

print(_in_mask(masks, 49, 66, 33))

{'001716': True, '013136': False, '073720': False, '125909': False, '131520': False, '156999': False, '176064': False, '182355': False, '216192': False, '275836': False, '291082': False, '414207': False, '426426': False, '447647': False, '450767': False, '477522': False, '525931': False, '532911': False, '533294': False, '534482': False, '707656': False, '752691': False, '779062': False, '791103': False, '831650': False, '877949': False, '911981': False, '954861': False}


### Task

In [3]:
def _is_task(task, name):
    if task == "gonogo" and task == name:
        return True
    elif task == "conscious" and task == name:
        return True
    elif task == "nonconscious" and task == name:
        return True
    elif task in ["workingmemSB", "workingmemMB"] and task == "workingmem":
        return True
    return False

### Operator

In [1]:
operators = {
    "adam": {"^apines", "apines", "Pines^Adam", "^apine"},
    "katherine": {"kgrisanz", "^kgrisanz"},
    "monica": {"^mkullar", "mkullar", "^mkullat"},
    "norween": {"^nowreen", "nowreen"},
    "sarah": {"sarchang"},
    "melissa": {"mshiner"},
    "david": {"choidd", "chiodd"},
    "bailey": {"bholtgos"},
    "megan": {"mchesnut"},
}

def _is_operator(name):
    def wrap(operator):
        names = operators[name]
        return operator in names
    return wrap

### fMRI

In [51]:
valid = pd.read_csv("/Volumes/hd_4tb/raw/info.csv")

def _time_map(session):
    _map = {
        "s1": "000", "s2": "2MO", "s3": "6MO", "s4": "12MO", "s5": "24MO"
    }
    return _map[session]

def _get(row, item):
    return row[item]

def _fmri_path(row):
    project = _get(row, "project")
    subject = _get(row, "subject")
    time_session = _get(row, "time_session")
    task = _get(row, "task")
    return join(_input, project, _time_map(time_session), subject, task)

def nii_input(data, x, y, z, t, r = 2):
    return data[
        x - r : x + r + 1,
        y - r : y + r + 1,
        z - r : z + r + 1,
        t
    ]

### Train

In [50]:
_input = "/Volumes/hd_4tb/raw"

for x, y, z in gen_avail_volumes(data.shape, radius=5):
    for t in range(1, data.shape[-1] - 1):
        before = nii_input(data, x, y, z, t-1)
        after  = nii_input(data, x, y, z, t+1)
        grey_masks = _in_masks(data, x, y, z, masks)

for i, row in valid.iterrows():
    fmri_path = _fmri_path(row)
    fmri = join(fmri_path, "normalized.nii.gz")
    onsets = join(fmri_path, "onsets.csv")
    grey = join(fmri_path, "..", "structural", "gm_probseg.nii.gz")